Installing Tensorflow and depending libraries

In [5]:
!pip install tensorflow tensorflow-gpu

No Python at 'C:\Users\iammh\AppData\Local\Programs\Python\Python37-32\python.exe'


Importing Standard Dependencies

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random

Importing Tensorflow Dependencies

In [37]:
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Layer, Conv2D, MaxPooling2D, Dense, Input, Flatten
import tensorflow as tf

Setting GPU Growth

In [38]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Creating Folder Structures

In [6]:
POS_PATH = os.path.join("data", 'positive')
NEG_PATH = os.path.join("data", 'negative')
ANC_PATH = os.path.join("data", 'anchor')

In [10]:
os.makedirs(POS_PATH, exist_ok=True)
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

In [13]:
## Uncompressing the tar file
!tar -xf lfw.tgz

 Collecting Positives and Anchor Classes

In [16]:
for directory in os.listdir('lfw'):
    for files in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, files)
        NEW_PATH = os.path.join(NEG_PATH, files)
        os.replace(EX_PATH, NEW_PATH)

In [6]:
#Importing UUID library to generate unique names for the images
import uuid

In [7]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()         # capturing the frame (Actual Image)

# Cutting frame to 250px by 250px
    frame = frame[120:120+250,200:200+250,:]

    #Collect Anchors
    if cv2.waitKey(1) & 0xFF == ord('a'):
        #Generating a unique name for the image
        anc_img_name = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        #Saving the image
        cv2.imwrite(anc_img_name, frame)
    #Collecting Positives
    if cv2.waitKey(1) & 0xFF == ord('p'):
        pos_img_name = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        #Saving the image
        cv2.imwrite(pos_img_name, frame)
    #Show image back to screen

    cv2.imshow('Image Collection', frame)

#Braking ties with the camera

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Releasing the webcam
cap.release()
cv2.destroyAllWindows()

## Load and preprocess the images

In [7]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(300)

In [8]:
dir_test = anchor.as_numpy_iterator()

In [9]:
dir_test.next()

b'data\\anchor\\8fa8534b-117d-11ed-8649-6c02e06ff1a3.jpg'

In [10]:
## Preprocessing the images
def preprocess(file_path):
    # Reading image from the file path
    byte_img = tf.io.read_file(file_path)
    # Loading the image
    img = tf.io.decode_jpeg(byte_img)
    # Preprocessing the image - Resizing to (100,100,3)
    img = tf.image.resize(img, (100,100))
    # Scaling image to be between 0 and 1
    img = img/255.0
    # Returning the image
    return img

In [11]:
img = preprocess('data\\anchor\\76e663d5-117d-11ed-8932-6c02e06ff1a3.jpg')

In [12]:
# (anchor, positive) => 1,1,1,1,1
# (anchor, positive) => 0,0,0,0,0
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [13]:
samples = data.as_numpy_iterator()

In [14]:
example = samples.next()

In [15]:
example

(b'data\\anchor\\e5f0658b-117d-11ed-9b4f-6c02e06ff1a3.jpg',
 b'data\\positive\\5f50defa-117f-11ed-8542-6c02e06ff1a3.jpg',
 1.0)

## Build Train and Test Partitions

In [16]:
def preprocess_twin(input_img, validation_img, label):
    return preprocess(input_img), preprocess(validation_img), label

In [17]:
res = preprocess_twin(*example)

In [18]:
res[2]

1.0

In [19]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
# Caching images to memory
data = data.cache()
# Shuffling the data
data = data.shuffle(buffer_size=1024)

In [20]:
# Training Partition
# Grabbing certain percentage of the data for training
train_data = data.take(round(len(data)*.7))
# Creating batches of 16 images
train_data = train_data.batch(16)
# Preprocessing next set of images
train_data = train_data.prefetch(0)

In [21]:
# Validation Partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(0)

# Model Engineering

## Embedding Layer

In [22]:
def make_embedding():
    inp = Input(shape=(100,100,3), name='input_image')

    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    #Second block
    c2 = Conv2D(128,(7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64,(2,2), padding="same")(c2)

    # Third block
    c3 = Conv2D(128,(4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64,(2,2), padding="same")(c3)

    # Fourth block
    c4 = Conv2D(256,(4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)

    return Model(inputs=[inp] , outputs=[d1], name='embedding')

In [23]:
embedding = make_embedding()

In [24]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 91, 91, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0 

## Build Distance Layer

In [25]:
# Siamese L1 Distance class
class L1Dist(Layer):
    # Init method - inheritance from Layer class
    def __init__(self, **kwargs):
        super().__init__()

    # Similarity Calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [26]:
l1 = L1Dist()

Make Siamese Model

In [27]:
def make_siamese_model():

    # Anchor image input in the network
    input_image = Input(shape=(100,100,3), name='input_img')
    # Validation image input in the network
    validation_image = Input(shape=(100,100,3), name='validation_img')

    # combine siamese distance layer with the embedding layer
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    # Classification layer
    classifier = Dense(1, activation='sigmoid')(distances)
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='siameseNetwork')

In [28]:
siamese_model = make_siamese_model()

In [29]:
siamese_model.summary()

Model: "siameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
distance (L1Dist)               (None, 4096)         0           embedding[0][0]     

Setup loss and optimizer

In [30]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [31]:
optimizer = tf.keras.optimizers.Adam(1e-4) # 0.0001

Establish Checkpoints

In [32]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(opt=optimizer, siamese_model=siamese_model)

## Build Train Step Function

In [33]:
@tf.function
def train_step(batch):

    # Record all of our operations
    with tf.GradientTape() as tape:
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get the labels
        y = batch[2]
        # Forward pass
        y_pred = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, y_pred)

        # Calculating gradients
        grad = tape.gradient(loss, siamese_model.trainable_variables)

        # Calculating updated weights and apply to siamese model
        optimizer.apply_gradients(zip(grad, siamese_model.trainable_variables))
        return loss

Build training loop

In [34]:
def train(t_data, t_epoch):
    # Loop through epochs
    for epoch in range(1, t_epoch+1):
        print('\n EPOCH {}/{}'.format(epoch, t_epoch))
        progbar = tf.keras.utils.Progbar(len(t_data))

        # Loop through each batch
        for idx, batch in enumerate(t_data):
            # Run train step here
            train_step(batch)
            progbar.update(idx+1)
        # Save checkpoints
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

## Train the Model on colab and then downloading the training checkpoints and siamesemodel.h5 file

In [42]:
# Reload model
model = tf.keras.models.load_model('siamesemodel.h5',
                                   custom_objects={'L1Dist':L1Dist})

IndexError: list index out of range